In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint


In [2]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2)  # 20% for validation

train_generator = train_datagen.flow_from_directory(
    r'Rock-Paper-Scissors\train',  # Replace with the path to your dataset
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    subset='training')

validation_generator = train_datagen.flow_from_directory(
    r'Rock-Paper-Scissors\test',  # Replace with the path to your dataset
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    subset='validation')

class_indices = train_generator.class_indices
class_labels = {v: k for k, v in class_indices.items()}
print("Class Indices:", class_indices)
print("Class Labels:", class_labels)

Found 2016 images belonging to 3 classes.
Found 72 images belonging to 3 classes.
Class Indices: {'paper': 0, 'rock': 1, 'scissors': 2}
Class Labels: {0: 'paper', 1: 'rock', 2: 'scissors'}


In [3]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(3, activation='softmax')  # 3 classes for rock, paper, scissors
])


c:\Users\Ramana\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [4]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [5]:
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('best_model_rps.keras', save_best_only=True, monitor='val_loss', mode='min')


In [ ]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=10,
    callbacks=[early_stopping, model_checkpoint]
)


In [ ]:
loss, accuracy = model.evaluate(validation_generator)
print(f'Validation Loss: {loss}')
print(f'Validation Accuracy: {accuracy}')


3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 125ms/step - accuracy: 0.9705 - loss: 0.0764
Validation Loss: 0.07199951261281967
Validation Accuracy: 0.9722222089767456


In [ ]:
model.save('Final.h5')